In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import subprocess
from subprocess import check_output
try:
    print(check_output(["ls", "../v2"]).decode("utf8"))
except subprocess.CalledProcessError as e:
    raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))
# Any results you write to the current directory are saved as output.

frame
video



In [2]:
import tensorflow as tf

In [3]:
# now, let's read the frame-level data
# due to execution time, we're only going to read the first video
video_files = os.listdir("../v2/video/")
frame_files = os.listdir("../v2/frame/")

# Global variables
num_labels = 4716

In [4]:
def createTargetVec(labels):
    out = np.zeros((1, num_labels))
    for label in labels:
        out[0,label] = 1
    return out

In [5]:
def get_dataset(frames, samples=5, k=0):
    rgb_input = np.empty((samples, 100, 1024))
    audio_input = np.empty((samples, 100, 128))
    label_output = np.empty((samples, num_labels))
    sess = tf.InteractiveSession()
    for example in tf.python_io.tf_record_iterator(frames):        
        tf_seq_example = tf.train.SequenceExample.FromString(example)
        labels = tf_seq_example.context.feature['labels'].int64_list.value
        rgb_frame = np.zeros((100, 1024))
        audio_frame = np.zeros((100, 128))
        for i in range(100):
            rgb_frame[i] = tf.cast(tf.decode_raw(
                   tf_seq_example.feature_lists.feature_list['rgb'].feature[i].bytes_list.value[0],tf.uint8)
                          ,tf.float32).eval()
            audio_frame[i] = tf.cast(tf.decode_raw(
                    tf_seq_example.feature_lists.feature_list['audio'].feature[i].bytes_list.value[0],tf.uint8)
                           ,tf.float32).eval()
#         print("rgb_frame: ", rgb_frame)
        rgb_input[k] = rgb_frame
#         print("rgb_input: ", rgb_input)
        audio_input[k] = audio_frame
        label_output[k] = createTargetVec(labels[:])
        k += 1
        progress = (k / samples) * 100
        if int(progress) % 10 == 0:
            print("Progress", progress, "%")
        if k >= samples:
            break
    sess.close()
    return audio_input, rgb_input, label_output

In [6]:
# video_lvl_record = "../v2/video/train-1.tfrecord"
frame_lvl_records = []
frame_lvl_record_1 = "../v2/frame/train4f.tfrecord"
frame_lvl_record_2 = "../v2/frame/testZZ.tfrecord"
frame_lvl_records.append(frame_lvl_record_1)
frame_lvl_records.append(frame_lvl_record_2)

In [7]:
len(frame_lvl_record_1)

28

In [8]:
len(frame_lvl_record_2)

27

In [9]:
audio_input, rgb_input, label_output = get_dataset(frame_lvl_record_1)

X_train_rgb, X_test_rgb, _, _ = train_test_split(rgb_input, label_output, test_size=0.2, random_state=42)
X_train_audio, X_test_audio, y_train, y_test = train_test_split(audio_input, label_output, test_size=0.2, random_state=42)

Progress 20.0 %
Progress 40.0 %
Progress 60.0 %
Progress 80.0 %
Progress 100.0 %


In [10]:
print(X_train_rgb.shape)
print(X_test_rgb.shape)
print(X_train_audio.shape)
print(X_test_audio.shape)

(4, 100, 1024)
(1, 100, 1024)
(4, 100, 128)
(1, 100, 128)


In [11]:
# (rgb) 1024 + (audio) 128 = (frame) 1152
x_train_np = np.concatenate((X_train_rgb, X_train_audio), axis=2)
x_test_np = np.concatenate((X_test_rgb, X_test_audio), axis=2)

In [12]:
print(x_train_np.shape)
print(x_test_np.shape)

(4, 100, 1152)
(1, 100, 1152)


## Now were ready to create our model
****
** Since we're given sequential data, most likely I'll use an LSTM**

In [13]:
import keras
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.merge import dot, concatenate
from keras.layers.recurrent import LSTM
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.layers import Input, Dense, Dropout, Bidirectional, Add, GlobalMaxPooling1D

Using TensorFlow backend.


### Frame Model
*** ***

In [14]:
X1 = Input(shape=(100,1152),name='frame')
fc_1 = Dense(2304,activation='relu',name='fc_1')(X1)
lstm_1 = LSTM(2304, return_sequences=True, go_backwards=False, name='lstm_1')(fc_1)

# First fast Merge connection
merge_1 = Add(name='merge_1')([fc_1, lstm_1])
fc_2 = Dense(2304,activation='relu',name='fc_2')(merge_1)
lstm_2 = LSTM(2304, return_sequences=True, go_backwards=True, name='lstm_2')(fc_2)

# second fast Merge connection
merge_2 = Add(name='merge_2')([fc_2, lstm_2])
fc_3 = Dense(2304,activation='relu',name='fc_3')(merge_2)
lstm_3 = LSTM(2304, return_sequences=True, go_backwards=False, name='lstm_3')(fc_3)

# third fast Merge connection
merge_3 = Add(name='merge_3')([fc_3, lstm_3])
fc_4 = Dense(2304,activation='relu',name='fc_4')(merge_3)
lstm_4 = LSTM(2304, return_sequences=True, go_backwards=True, name='lstm_4')(fc_4)

# Pooling
pool = GlobalMaxPooling1D(name='global_max_pool')(lstm_4)
# FC_2048
fc_2048 = Dense(2048, activation='relu',name='fc_2048')(pool)
# Softmax
output = Dense(4716, activation='softmax',name='output')(fc_2048)

In [15]:
# Complete Model Diagram
frame_model = Model(inputs=[X1],outputs=[output])
frame_model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'] )

In [16]:
plot_model(frame_model,to_file='frame_model.png',show_shapes=True)

In [17]:
frame_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
frame (InputLayer)              (None, 100, 1152)    0                                            
__________________________________________________________________________________________________
fc_1 (Dense)                    (None, 100, 2304)    2656512     frame[0][0]                      
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 100, 2304)    42476544    fc_1[0][0]                       
__________________________________________________________________________________________________
merge_1 (Add)                   (None, 100, 2304)    0           fc_1[0][0]                       
                                                                 lstm_1[0][0]                     
__________

In [20]:
frame_model.fit(x_train_np, y_train, validation_data=(x_test_np, y_test), epochs=10, batch_size=5)

In [ ]:
# evaluate the model
# scores = frame_model.evaluate(x_train_np, y_train, verbose=0)
# print("")
# print("%s: %.2f%%" % (frame_model.metrics_names[1], scores[1]*100))

In [ ]:
# evaluate loaded model on test data
frame_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
_score = frame_model.evaluate(x_train_np, y_train, verbose=0)
print("%s: %.2f%%" % (frame_model.metrics_names[1], _score[1]*100))